In [3]:
import numpy as np
import tensorflow as tf

In [9]:
data1 = np.load('train_data.npz')
train_input = data1['inputs']
train_target = data1['targets']

data2 = np.load('validation_data.npz')
validation_input = data2['inputs']
validation_target = data2['targets']

data3 = np.load('test_data.npz')
test_input = data3['inputs']
test_target = data3['targets']

In [25]:
hidden_layer_size = 50
output_layer_size = 3

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_layer_size)
])

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

model.compile(
    optimizer = optimizer,
    loss = 'mse',
    metrics = ['mae']
)

In [26]:
early_stop = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

model.fit(
    train_input,
    train_target,
    validation_data=(validation_input, validation_target),
    epochs=100,
    batch_size=64,
    callbacks=early_stop,
    verbose=1
)

Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 1.1193 - mae: 0.9123 - val_loss: 0.9036 - val_mae: 0.8188
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.8195 - mae: 0.7807 - val_loss: 0.6785 - val_mae: 0.7038
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6514 - mae: 0.6913 - val_loss: 0.5407 - val_mae: 0.6222
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5508 - mae: 0.6290 - val_loss: 0.4569 - val_mae: 0.5559
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4527 - mae: 0.5612 - val_loss: 0.4124 - val_mae: 0.5140
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4075 - mae: 0.5169 - val_loss: 0.3796 - val_mae: 0.4870
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3678 - mae: 0.4833 - val_loss: 0.3578 - val_mae: 0.4694
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3537 - mae: 0.4752 - val_loss: 0.3365 - val_mae: 0.4507
Epoch 9/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.31

In [27]:
y_pred = model.predict(test_input)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [36]:
import joblib
import pandas as pd
scaled_target = joblib.load('scaler_target.pkl')
y_pred_unscale = scaled_target.inverse_transform(y_pred)
y_pred_unscale = pd.DataFrame(y_pred_unscale, columns=['Kp_pred', 'Ki_pred', 'Kd_pred'])

y_actual = scaled_target.inverse_transform(test_target)
y_actual = pd.DataFrame(y_actual, columns=['Kp_actual', 'Ki_actual', 'Kd_actual'])

table = pd.concat([y_actual, y_pred_unscale], axis=1)
table

,Kp_actual,Ki_actual,Kd_actual,Kp_pred,Ki_pred,Kd_pred
0,20.0,100.0,2.0,18.001228,105.961533,3.093745
1,100.0,0.0,1.0,99.135910,4.737206,1.013824
2,20.0,20.0,4.0,15.119453,10.683281,3.382555
3,50.0,80.0,1.5,52.859703,103.626038,3.013631
4,30.0,0.0,0.5,31.182764,8.775063,2.063153
...,...,...,...,...,...,...
105,50.0,20.0,3.0,45.591629,16.212624,2.410883
106,40.0,0.0,4.5,41.392365,8.121010,3.844662
107,20.0,160.0,4.5,20.177723,155.994354,4.264398
108,60.0,200.0,4.5,60.357479,146.009308,2.759056


In [39]:
table['Kp_error'] = abs(table['Kp_actual'] - table['Kp_pred'])
table['Ki_error'] = abs(table['Ki_actual'] - table['Ki_pred'])
table['Kd_error'] = abs(table['Kd_actual'] - table['Kd_pred'])
table

,Kp_actual,Ki_actual,Kd_actual,Kp_pred,Ki_pred,Kd_pred,Kp_error,Ki_error,Kd_error
0,20.0,100.0,2.0,18.001228,105.961533,3.093745,1.998772,5.961533,1.093745
1,100.0,0.0,1.0,99.135910,4.737206,1.013824,0.864090,4.737206,0.013824
2,20.0,20.0,4.0,15.119453,10.683281,3.382555,4.880547,9.316719,0.617445
3,50.0,80.0,1.5,52.859703,103.626038,3.013631,2.859703,23.626038,1.513631
4,30.0,0.0,0.5,31.182764,8.775063,2.063153,1.182764,8.775063,1.563153
...,...,...,...,...,...,...,...,...,...
105,50.0,20.0,3.0,45.591629,16.212624,2.410883,4.408371,3.787376,0.589117
106,40.0,0.0,4.5,41.392365,8.121010,3.844662,1.392365,8.121010,0.655338
107,20.0,160.0,4.5,20.177723,155.994354,4.264398,0.177723,4.005646,0.235602
108,60.0,200.0,4.5,60.357479,146.009308,2.759056,0.357479,53.990692,1.740944


In [40]:
table.reset_index(drop=True)
pd.DataFrame.to_csv(table, 'output.csv')